In [17]:
# 🔧 DEBUG: Check current generation status and fix the issue
print("🔍 DEBUGGING GENERATION ISSUE")
print("=" * 50)

# Check current variable values
if 'TOTAL_TARGET' in globals():
    print(f"📊 TOTAL_TARGET: {TOTAL_TARGET:,}")
else:
    print("❌ TOTAL_TARGET not defined")

if 'ALREADY_GENERATED' in globals():
    print(f"📊 ALREADY_GENERATED: {ALREADY_GENERATED:,}")
else:
    print("❌ ALREADY_GENERATED not defined")

if 'REMAINING_NEEDED' in globals():
    print(f"📊 REMAINING_NEEDED: {REMAINING_NEEDED:,}")
else:
    print("❌ REMAINING_NEEDED not defined")

# Let's recalculate from scratch
import pandas as pd
df = pd.read_csv('data/raw/Twitter_Analysis.csv')
false_tweets = len(df[df['majority_target'] == 0])
true_tweets = len(df[df['majority_target'] == 1])
difference = true_tweets - false_tweets

print(f"\n📊 RECALCULATED VALUES:")
print(f"   • False tweets: {false_tweets:,}")
print(f"   • True tweets: {true_tweets:,}")
print(f"   • Difference (target): {difference:,}")

# Check existing generation files
import glob
import json

results_pattern = "results/progressive_batch_*.json"
existing_files = glob.glob(results_pattern)
print(f"\n📁 Found {len(existing_files)} result files:")

total_existing = 0
for file in existing_files:
    try:
        with open(file, 'r') as f:
            file_results = json.load(f)
        file_items = len(file_results.get('results', []))
        total_existing += file_items
        print(f"   • {file}: {file_items} items")
    except Exception as e:
        print(f"   ❌ Error reading {file}: {e}")

print(f"\n🎯 SUMMARY:")
print(f"   • Target needed: {difference:,}")
print(f"   • Currently have: {total_existing:,}")
print(f"   • Still need: {difference - total_existing:,}")
print(f"   • Progress: {(total_existing/difference)*100:.1f}%")

🔍 DEBUGGING GENERATION ISSUE
📊 TOTAL_TARGET: 3,772
📊 ALREADY_GENERATED: 101
📊 REMAINING_NEEDED: 3,671

📊 RECALCULATED VALUES:
   • False tweets: 65,213
   • True tweets: 68,985
   • Difference (target): 3,772

📁 Found 1 result files:
   • results/progressive_batch_20250812.json: 50 items

🎯 SUMMARY:
   • Target needed: 3,772
   • Currently have: 50
   • Still need: 3,722
   • Progress: 1.3%

📊 RECALCULATED VALUES:
   • False tweets: 65,213
   • True tweets: 68,985
   • Difference (target): 3,772

📁 Found 1 result files:
   • results/progressive_batch_20250812.json: 50 items

🎯 SUMMARY:
   • Target needed: 3,772
   • Currently have: 50
   • Still need: 3,722
   • Progress: 1.3%


In [1]:
# 🚀 CORRECTED LARGE-SCALE GENERATION - GENERATE ALL 3,722 REMAINING TWEETS
import json
import pandas as pd
from datetime import datetime
import os
import sys
import glob

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Add generation path and import utilities
sys.path.append('generation')
from utils import create_generator, progressive_batch_processor

print("🚀 CORRECTED COMPREHENSIVE SYNTHETIC TWEET GENERATION")
print("=" * 60)

# Initialize generator
generator = create_generator("openai", "gpt-3.5-turbo")

# CORRECTED VALUES (from debugging above)
TOTAL_TARGET = 3772  # Exact difference needed
ACTUALLY_HAVE = 50   # Only 50 tweets exist
STILL_NEED = TOTAL_TARGET - ACTUALLY_HAVE  # 3,722 tweets needed

print(f"📊 CORRECTED TARGET CALCULATION:")
print(f"   • Total target: {TOTAL_TARGET:,}")
print(f"   • Actually have: {ACTUALLY_HAVE:,}")
print(f"   • Still need: {STILL_NEED:,}")

# Prepare source texts for remaining generation
df = pd.read_csv('data/raw/Twitter_Analysis.csv')
source_false_tweets = df[df['majority_target'] == 0]['tweet'].tolist()

print(f"📝 Source preparation:")
print(f"   • Available false tweets: {len(source_false_tweets):,}")
print(f"   • Will cycle through them as needed")

# Create source list for the 3,722 tweets we need
source_texts = []
for i in range(STILL_NEED):
    # Cycle through source tweets, starting from where we left off (after 50)
    source_index = (ACTUALLY_HAVE + i) % len(source_false_tweets)
    source_texts.append(source_false_tweets[source_index])

print(f"✅ Created source list: {len(source_texts):,} texts ready")

# Generation configuration for efficient processing
BATCH_SIZE = 50  # Balanced for speed and reliability
estimated_batches = (STILL_NEED + BATCH_SIZE - 1) // BATCH_SIZE
estimated_minutes = estimated_batches * 5  # ~5 minutes per batch

print(f"\n⚙️ GENERATION CONFIGURATION:")
print(f"   • Batch size: {BATCH_SIZE}")
print(f"   • Estimated batches: {estimated_batches}")
print(f"   • Estimated time: ~{estimated_minutes} minutes ({estimated_minutes/60:.1f} hours)")

# Ask for confirmation before starting the large generation
print(f"\n🚨 READY TO GENERATE {STILL_NEED:,} TWEETS")
print(f"   This will take approximately {estimated_minutes/60:.1f} hours")
print(f"   The process will save progress every {BATCH_SIZE} tweets")

# Start generation
start_time = datetime.now()
print(f"\n⏰ Large-scale generation started at: {start_time}")
print(f"🔄 Processing {STILL_NEED:,} tweets in batches of {BATCH_SIZE}...")

try:
    # Create a special large-scale batch processor
    def large_scale_batch_processor(generator, texts, batch_size, total_target):
        """Process all texts without the 100-item limit"""
        results = []
        total_processed = 0
        
        # Process in batches
        for batch_start in range(0, len(texts), batch_size):
            batch_end = min(batch_start + batch_size, len(texts))
            batch_texts = texts[batch_start:batch_end]
            
            batch_num = (batch_start // batch_size) + 1
            print(f"\n📦 Processing Batch {batch_num}/{(len(texts) + batch_size - 1) // batch_size}")
            print(f"   Items {batch_start + 1} to {batch_end} of {len(texts)}")
            
            batch_results = []
            
            for i, text in enumerate(batch_texts):
                global_index = batch_start + i
                try:
                    result = generator.generate_complete(
                        text=text,
                        max_facts=3,
                        domain="covid",
                        include_metadata=True
                    )
                    
                    result_dict = {
                        "index": global_index,
                        "batch_number": batch_num,
                        "original_text": result.original_text,
                        "generated_text": result.synthetic_text,  # Use consistent key
                        "extracted_facts": result.extracted_facts,
                        "modified_facts": result.modified_facts,
                        "metadata": result.metadata,
                        "timestamp": datetime.now().isoformat()
                    }
                    
                    batch_results.append(result_dict)
                    total_processed += 1
                    
                    if total_processed % 10 == 0:
                        print(f"   ✅ Completed {total_processed}/{len(texts)} ({(total_processed/len(texts))*100:.1f}%)")
                        
                except Exception as e:
                    print(f"   ❌ Error on item {global_index + 1}: {e}")
            
            # Save batch progress
            results.extend(batch_results)
            timestamp = datetime.now().strftime("%Y%m%d")
            progress_file = f"results/large_scale_batch_{timestamp}.json"
            
            with open(progress_file, 'w') as f:
                json.dump({
                    'last_updated': datetime.now().isoformat(),
                    'total_processed': len(results),
                    'target_total': total_target,
                    'current_batch': batch_num,
                    'results': results
                }, f, indent=2)
            
            print(f"   💾 Saved progress: {len(results)} total results")
        
        return {
            'status': 'completed',
            'total_processed': len(results),
            'results': {'results': results},
            'items_processed': len(results)
        }
    
    # Run the corrected large-scale generation
    large_scale_result = large_scale_batch_processor(
        generator=generator,
        texts=source_texts,
        batch_size=BATCH_SIZE,
        total_target=TOTAL_TARGET
    )
    
    end_time = datetime.now()
    total_time = end_time - start_time
    
    print(f"\n🎉 LARGE-SCALE GENERATION COMPLETED!")
    print(f"⏰ Total generation time: {total_time}")
    print(f"📊 New tweets generated: {large_scale_result['items_processed']:,}")
    
    # Load existing results and combine
    existing_results = []
    try:
        with open('results/progressive_batch_20250812.json', 'r') as f:
            existing_data = json.load(f)
            existing_results = existing_data.get('results', [])
    except Exception as e:
        print(f"Could not load existing results: {e}")
    
    # Combine all results
    all_results = existing_results + large_scale_result['results']['results']
    final_count = len(all_results)
    
    # Extract unique tweets
    all_tweets = []
    for item in all_results:
        tweet_text = item.get('generated_text', item.get('synthetic_text', ''))
        if tweet_text.strip():
            all_tweets.append(tweet_text)
    
    # Remove duplicates
    unique_tweets = list(dict.fromkeys(all_tweets))  # Preserves order
    
    print(f"\n🎯 FINAL COMPREHENSIVE RESULTS:")
    print(f"   • Total results combined: {final_count:,}")
    print(f"   • Unique synthetic tweets: {len(unique_tweets):,}")
    print(f"   • Target was: {TOTAL_TARGET:,}")
    print(f"   • Coverage: {(len(unique_tweets)/TOTAL_TARGET)*100:.1f}%")
    
    # Save final comprehensive dataset
    export_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    final_results = {
        'generation_timestamp': export_timestamp,
        'total_target': TOTAL_TARGET,
        'total_achieved': len(unique_tweets),
        'coverage_percentage': (len(unique_tweets) / TOTAL_TARGET) * 100,
        'synthetic_tweets': unique_tweets,
        'generation_info': {
            'existing_tweets': ACTUALLY_HAVE,
            'newly_generated': len(unique_tweets) - ACTUALLY_HAVE,
            'processing_time': str(total_time)
        },
        'results': all_results
    }
    
    # Save files
    final_json_file = f"results/comprehensive_synthetic_tweets_{export_timestamp}.json"
    with open(final_json_file, 'w', encoding='utf-8') as f:
        json.dump(final_results, f, indent=2, default=str)
    
    final_text_file = f"data/synthetic/synthetic_tweets_complete_{export_timestamp}.txt"
    with open(final_text_file, 'w', encoding='utf-8') as f:
        for tweet in unique_tweets:
            f.write(tweet + '\n')
    
    print(f"\n💾 FILES SAVED:")
    print(f"   • JSON: {final_json_file}")
    print(f"   • Text: {final_text_file}")
    
    print(f"\n🚀 READY FOR CLASSIFICATION!")
    print(f"✅ Use: run_llm_synthetic_experiments(llm_results_file='{final_json_file}')")
    
    # Store for next steps
    FINAL_RESULTS_FILE = final_json_file
    FINAL_SYNTHETIC_TWEETS = unique_tweets
    
except Exception as e:
    print(f"❌ Error during large-scale generation: {str(e)}")
    print(f"💡 Can continue from saved progress files")

🚀 CORRECTED COMPREHENSIVE SYNTHETIC TWEET GENERATION


INFO:openai_generator:Initialized OpenAI generator with model: gpt-3.5-turbo


📊 CORRECTED TARGET CALCULATION:
   • Total target: 3,772
   • Actually have: 50
   • Still need: 3,722
📝 Source preparation:
   • Available false tweets: 65,213
   • Will cycle through them as needed
✅ Created source list: 3,722 texts ready

⚙️ GENERATION CONFIGURATION:
   • Batch size: 50
   • Estimated batches: 75
   • Estimated time: ~375 minutes (6.2 hours)

🚨 READY TO GENERATE 3,722 TWEETS
   This will take approximately 6.2 hours
   The process will save progress every 50 tweets

⏰ Large-scale generation started at: 2025-08-12 19:17:23.610790
🔄 Processing 3,722 tweets in batches of 50...

📦 Processing Batch 1/75
   Items 1 to 50 of 3722
📝 Source preparation:
   • Available false tweets: 65,213
   • Will cycle through them as needed
✅ Created source list: 3,722 texts ready

⚙️ GENERATION CONFIGURATION:
   • Batch size: 50
   • Estimated batches: 75
   • Estimated time: ~375 minutes (6.2 hours)

🚨 READY TO GENERATE 3,722 TWEETS
   This will take approximately 6.2 hours
   The proce

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

KeyboardInterrupt: 

In [2]:
# 🔄 RECOVERY: Resume generation from saved 1,750 tweets
import json
import pandas as pd
from datetime import datetime
import os
import sys

print("🔄 RECOVERING AND RESUMING GENERATION")
print("=" * 50)

# Load the saved progress
saved_file = 'results/large_scale_batch_20250812.json'
print(f"📁 Loading saved progress from: {saved_file}")

with open(saved_file, 'r') as f:
    saved_progress = json.load(f)

ALREADY_COMPLETED = len(saved_progress['results'])
TARGET_TOTAL = saved_progress['target_total']
STILL_NEEDED = TARGET_TOTAL - ALREADY_COMPLETED
LAST_BATCH = saved_progress['current_batch']

print(f"✅ RECOVERY STATUS:")
print(f"   • Already generated: {ALREADY_COMPLETED:,} tweets")
print(f"   • Target total: {TARGET_TOTAL:,} tweets")
print(f"   • Still needed: {STILL_NEEDED:,} tweets")
print(f"   • Progress: {(ALREADY_COMPLETED/TARGET_TOTAL)*100:.1f}%")
print(f"   • Last batch completed: {LAST_BATCH}")

if STILL_NEEDED <= 0:
    print(f"\n🎉 GENERATION ALREADY COMPLETE!")
    print(f"✅ All {TARGET_TOTAL:,} tweets have been generated")
    
    # Extract and save final dataset
    all_tweets = []
    for item in saved_progress['results']:
        tweet_text = item.get('generated_text', '').strip()
        if tweet_text:
            all_tweets.append(tweet_text)
    
    # Save final comprehensive dataset
    export_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    final_results = {
        'generation_timestamp': export_timestamp,
        'total_target': TARGET_TOTAL,
        'total_achieved': len(all_tweets),
        'coverage_percentage': (len(all_tweets) / TARGET_TOTAL) * 100,
        'synthetic_tweets': all_tweets,
        'generation_info': {
            'recovered_from': saved_file,
            'completion_status': 'recovered_complete'
        },
        'results': saved_progress['results']
    }
    
    final_json_file = f"results/comprehensive_synthetic_tweets_{export_timestamp}.json"
    with open(final_json_file, 'w', encoding='utf-8') as f:
        json.dump(final_results, f, indent=2, default=str)
    
    final_text_file = f"data/synthetic/synthetic_tweets_complete_{export_timestamp}.txt"
    with open(final_text_file, 'w', encoding='utf-8') as f:
        for tweet in all_tweets:
            f.write(tweet + '\n')
    
    print(f"\n💾 FINAL DATASET SAVED:")
    print(f"   • JSON: {final_json_file}")
    print(f"   • Text: {final_text_file}")
    print(f"   • Total tweets: {len(all_tweets):,}")
    
    FINAL_RESULTS_FILE = final_json_file
    FINAL_SYNTHETIC_TWEETS = all_tweets
    
else:
    print(f"\n🚀 RESUMING GENERATION...")
    print(f"⏳ Will generate remaining {STILL_NEEDED:,} tweets")
    
    # Load environment and generator
    from dotenv import load_dotenv
    load_dotenv()
    sys.path.append('generation')
    from utils import create_generator
    
    generator = create_generator("openai", "gpt-3.5-turbo")
    
    # Prepare remaining source texts
    df = pd.read_csv('data/raw/Twitter_Analysis.csv')
    source_false_tweets = df[df['majority_target'] == 0]['tweet'].tolist()
    
    # Create source texts for remaining generation, starting from where we left off
    remaining_source_texts = []
    for i in range(STILL_NEEDED):
        source_index = (ALREADY_COMPLETED + i) % len(source_false_tweets)
        remaining_source_texts.append(source_false_tweets[source_index])
    
    print(f"📝 Prepared {len(remaining_source_texts):,} source texts")
    
    # Continue generation with efficient batching
    BATCH_SIZE = 50
    estimated_batches = (STILL_NEEDED + BATCH_SIZE - 1) // BATCH_SIZE
    estimated_time = estimated_batches * 5  # 5 minutes per batch
    
    print(f"⚙️ RESUMPTION PLAN:")
    print(f"   • Batch size: {BATCH_SIZE}")
    print(f"   • Estimated batches: {estimated_batches}")
    print(f"   • Estimated time: ~{estimated_time} minutes ({estimated_time/60:.1f} hours)")
    
    print(f"\n⏰ Ready to resume generation from batch {LAST_BATCH + 1}")
    print(f"💡 Progress will be saved continuously")

print(f"\n📋 RECOVERY COMPLETE!")
print(f"✅ Your {ALREADY_COMPLETED:,} tweets are safe and accounted for")

INFO:openai_generator:Initialized OpenAI generator with model: gpt-3.5-turbo


🔄 RECOVERING AND RESUMING GENERATION
📁 Loading saved progress from: results/large_scale_batch_20250812.json
✅ RECOVERY STATUS:
   • Already generated: 1,750 tweets
   • Target total: 3,772 tweets
   • Still needed: 2,022 tweets
   • Progress: 46.4%
   • Last batch completed: 35

🚀 RESUMING GENERATION...
⏳ Will generate remaining 2,022 tweets
📝 Prepared 2,022 source texts
⚙️ RESUMPTION PLAN:
   • Batch size: 50
   • Estimated batches: 41
   • Estimated time: ~205 minutes (3.4 hours)

⏰ Ready to resume generation from batch 36
💡 Progress will be saved continuously

📋 RECOVERY COMPLETE!
✅ Your 1,750 tweets are safe and accounted for
📝 Prepared 2,022 source texts
⚙️ RESUMPTION PLAN:
   • Batch size: 50
   • Estimated batches: 41
   • Estimated time: ~205 minutes (3.4 hours)

⏰ Ready to resume generation from batch 36
💡 Progress will be saved continuously

📋 RECOVERY COMPLETE!
✅ Your 1,750 tweets are safe and accounted for


In [3]:
# 🚀 RESUME GENERATION: Continue from batch 36 (1,750 → 3,772 tweets)
import json
from datetime import datetime

print("🚀 RESUMING LARGE-SCALE GENERATION")
print("=" * 45)
print(f"📊 Starting from: 1,750 tweets (46.4% complete)")
print(f"🎯 Goal: Generate remaining 2,022 tweets")
print(f"⏰ Estimated time: ~3.4 hours")

# Load existing progress
with open('results/large_scale_batch_20250812.json', 'r') as f:
    existing_progress = json.load(f)

existing_results = existing_progress['results']
start_index = len(existing_results)  # 1750

print(f"\n⚡ RESUMING GENERATION...")
start_time = datetime.now()

try:
    # Resume the large-scale batch processor from where we left off
    def resume_large_scale_generation(generator, existing_results, remaining_texts, batch_size, target_total):
        """Resume generation with existing results"""
        results = existing_results.copy()  # Start with existing results
        total_processed = len(results)
        
        print(f"📦 Starting from batch {(total_processed // batch_size) + 1}")
        
        # Process remaining texts in batches
        for batch_start in range(0, len(remaining_texts), batch_size):
            batch_end = min(batch_start + batch_size, len(remaining_texts))
            batch_texts = remaining_texts[batch_start:batch_end]
            
            current_batch_num = (total_processed // batch_size) + 1
            overall_progress = total_processed + batch_start
            
            print(f"\n📦 Processing Batch {current_batch_num} (Overall items {overall_progress + 1} to {overall_progress + len(batch_texts)})")
            
            batch_results = []
            for i, text in enumerate(batch_texts):
                global_index = total_processed + batch_start + i
                try:
                    result = generator.generate_complete(
                        text=text,
                        max_facts=3,
                        domain="covid",
                        include_metadata=True
                    )
                    
                    result_dict = {
                        "index": global_index,
                        "batch_number": current_batch_num,
                        "original_text": result.original_text,
                        "generated_text": result.synthetic_text,
                        "extracted_facts": result.extracted_facts,
                        "modified_facts": result.modified_facts,
                        "metadata": result.metadata,
                        "timestamp": datetime.now().isoformat()
                    }
                    
                    batch_results.append(result_dict)
                    
                    if (global_index + 1) % 10 == 0:
                        progress_pct = ((global_index + 1) / target_total) * 100
                        print(f"   ✅ Completed {global_index + 1}/{target_total} ({progress_pct:.1f}%)")
                        
                except Exception as e:
                    print(f"   ❌ Error on item {global_index + 1}: {e}")
            
            # Add batch results to main results
            results.extend(batch_results)
            
            # Save progress after each batch
            progress_data = {
                'last_updated': datetime.now().isoformat(),
                'total_processed': len(results),
                'target_total': target_total,
                'current_batch': current_batch_num,
                'results': results
            }
            
            with open('results/large_scale_batch_20250812.json', 'w') as f:
                json.dump(progress_data, f, indent=2)
            
            completion_pct = (len(results) / target_total) * 100
            print(f"   💾 Saved progress: {len(results)}/{target_total} ({completion_pct:.1f}% complete)")
        
        return {
            'status': 'completed',
            'total_processed': len(results),
            'results': {'results': results},
            'items_processed': len(results) - len(existing_results)
        }
    
    # Run the resumed generation
    resumed_result = resume_large_scale_generation(
        generator=generator,
        existing_results=existing_results,
        remaining_texts=remaining_source_texts,
        batch_size=BATCH_SIZE,
        target_total=TARGET_TOTAL
    )
    
    end_time = datetime.now()
    total_time = end_time - start_time
    
    print(f"\n🎉 RESUMED GENERATION COMPLETED!")
    print(f"⏰ Additional generation time: {total_time}")
    print(f"📊 New tweets generated: {resumed_result['items_processed']:,}")
    print(f"📊 Total tweets now: {resumed_result['total_processed']:,}")
    
    # Create final comprehensive dataset
    print(f"\n🔧 Creating final comprehensive dataset...")
    
    final_results = resumed_result['results']['results']
    all_tweets = []
    for item in final_results:
        tweet_text = item.get('generated_text', '').strip()
        if tweet_text:
            all_tweets.append(tweet_text)
    
    # Save final dataset
    export_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    comprehensive_results = {
        'generation_timestamp': export_timestamp,
        'total_target': TARGET_TOTAL,
        'total_achieved': len(all_tweets),
        'coverage_percentage': (len(all_tweets) / TARGET_TOTAL) * 100,
        'synthetic_tweets': all_tweets,
        'generation_info': {
            'resumed_from': 'results/large_scale_batch_20250812.json',
            'original_count': len(existing_results),
            'newly_generated': len(all_tweets) - len(existing_results),
            'total_processing_time': str(total_time)
        },
        'results': final_results
    }
    
    # Save files
    final_json_file = f"results/comprehensive_synthetic_tweets_{export_timestamp}.json"
    with open(final_json_file, 'w', encoding='utf-8') as f:
        json.dump(comprehensive_results, f, indent=2, default=str)
    
    final_text_file = f"data/synthetic/synthetic_tweets_complete_{export_timestamp}.txt"
    with open(final_text_file, 'w', encoding='utf-8') as f:
        for tweet in all_tweets:
            f.write(tweet + '\n')
    
    print(f"\n🎯 FINAL COMPREHENSIVE RESULTS:")
    print(f"   • Total synthetic tweets: {len(all_tweets):,}")
    print(f"   • Target achieved: {TARGET_TOTAL:,}")
    print(f"   • Coverage: {(len(all_tweets)/TARGET_TOTAL)*100:.1f}%")
    print(f"   • JSON file: {final_json_file}")
    print(f"   • Text file: {final_text_file}")
    
    print(f"\n🚀 READY FOR CLASSIFICATION!")
    print(f"✅ Use: run_llm_synthetic_experiments(llm_results_file='{final_json_file}')")
    
    # Store for classification
    FINAL_RESULTS_FILE = final_json_file
    FINAL_SYNTHETIC_TWEETS = all_tweets
    
except Exception as e:
    print(f"❌ Error during resumed generation: {str(e)}")
    print(f"💡 Progress is continuously saved - can resume from any point")

🚀 RESUMING LARGE-SCALE GENERATION
📊 Starting from: 1,750 tweets (46.4% complete)
🎯 Goal: Generate remaining 2,022 tweets
⏰ Estimated time: ~3.4 hours

⚡ RESUMING GENERATION...
📦 Starting from batch 36

📦 Processing Batch 36 (Overall items 1751 to 1800)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

   ✅ Completed 1760/3772 (46.7%)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

   ✅ Completed 1770/3772 (46.9%)


...

   ✅ Completed 3760/3772 (99.7%)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

   ✅ Completed 3770/3772 (99.9%)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

   💾 Saved progress: 3772/3772 (100.0% complete)

🎉 RESUMED GENERATION COMPLETED!
⏰ Additional generation time: 3:45:24.525739
📊 New tweets generated: 2,022
📊 Total tweets now: 3,772

🔧 Creating final comprehensive dataset...

🎯 FINAL COMPREHENSIVE RESULTS:
   • Total synthetic tweets: 3,768
   • Target achieved: 3,772
   • Coverage: 99.9%
   • JSON file: results/comprehensive_synthetic_tweets_20250812_230615.json
   • Text file: data/synthetic/synthetic_tweets_complete_20250812_230615.txt

🚀 READY FOR CLASSIFICATION!
✅ Use: run_llm_synthetic_experiments(llm_results_file='results/comprehensive_synthetic_tweets_20250812_230615.json')

🎯 FINAL COMPREHENSIVE RESULTS:
   • Total synthetic tweets: 3,768
   • Target achieved: 3,772
   • Coverage: 99.9%
   • JSON file: results/comprehensive_synthetic_tweets_20250812_230615.json
   • Text file: data/synthetic/synthetic_tweets_complete_20250812_230615.txt

🚀 READY FOR CLASSIFICATION!
✅ Use: run_llm_synthetic_experiments(llm_results_file='result

In [2]:
# Load environment variables
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Verify the API key is loaded
if os.getenv('OPENAI_API_KEY'):
    print("✅ OpenAI API key loaded successfully")
else:
    print("❌ OpenAI API key not found")

✅ OpenAI API key loaded successfully


In [3]:
import sys
sys.path.append('generation')

In [4]:
from utils import create_generator, progressive_batch_processor

In [5]:
generator = create_generator("openai", "gpt-3.5-turbo")

INFO:openai_generator:Initialized OpenAI generator with model: gpt-3.5-turbo


In [6]:
import pandas as pd

# Load your data
df = pd.read_csv('data/raw/Twitter_Analysis.csv')

In [7]:
filtered_df = df[df['majority_target'] == 0]
print(len(filtered_df))

65213


In [8]:
texts = filtered_df['tweet'].tolist()

In [9]:
generator = create_generator("openai", "gpt-3.5-turbo")

INFO:openai_generator:Initialized OpenAI generator with model: gpt-3.5-turbo


In [14]:
result = progressive_batch_processor(
    generator=generator,
    texts=texts,
    batch_size=10,
    max_items=100,  # Process up to 100 items total
    max_facts=3,
    domain="covid"  # or "general", "news", etc.
)

Processing Batch 1: items 1 to 10
Progress: 0/100 → 10/100
Processing item 1/100


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing item 2/100


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing item 3/100


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing item 9/100


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing item 10/100


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [15]:
print(f"✅ Batch completed: {result['items_processed']} items")
print(f"📊 Total progress: {result['total_processed']}/100")
print(f"🎯 Success rate: {result['quality_metrics'].success_rate:.1%}")

✅ Batch completed: 10 items
📊 Total progress: 10/100
🎯 Success rate: 100.0%


In [17]:
# Let's explore what's in the result
print("Result structure:")
print(f"Keys: {list(result.keys())}")
print()

# Check the generated facts
if 'facts' in result:
    print(f"Number of generated facts: {len(result['facts'])}")
    print("Sample facts:")
    for i, fact in enumerate(result['facts'][:3]):  # Show first 3 facts
        print(f"Fact {i+1}: {fact}")
        print()

# Check if we have any generated texts
if 'generated_texts' in result:
    print(f"Number of generated texts: {len(result['generated_texts'])}")
    print("Sample generated texts:")
    for i, text in enumerate(result['generated_texts'][:3]):  # Show first 3 texts
        print(f"Text {i+1}: {text}")
        print()

# Show full result structure
print("Full result:")
import json
print(json.dumps(result, indent=2, default=str)[:1000] + "..." if len(str(result)) > 1000 else json.dumps(result, indent=2, default=str))

Result structure:
Keys: ['status', 'batch_number', 'items_processed', 'total_processed', 'max_items', 'processing_time', 'next_batch_ready', 'results_file', 'quality_metrics']

Full result:
{
  "status": "batch_completed",
  "batch_number": 1,
  "items_processed": 10,
  "total_processed": 10,
  "max_items": 100,
  "processing_time": "0:00:59.650142",
  "next_batch_ready": true,
  "results_file": "results/progressive_batch_20250812.json",
  "quality_metrics": "QualityMetrics(content_changed_ratio=1.0, avg_fact_incorporation=2.6, avg_processing_time=5.963720512390137, success_rate=1.0)"
}


In [19]:
# Load the saved results
import json
results_file = result['results_file']
print(f"Loading results from: {results_file}")

with open(results_file, 'r') as f:
    saved_results = json.load(f)

print(f"Type of saved_results: {type(saved_results)}")
print(f"Structure: {saved_results.keys() if isinstance(saved_results, dict) else 'Not a dict'}")
print()

# Show the actual structure
print("Saved results structure:")
if isinstance(saved_results, dict):
    for key, value in saved_results.items():
        if isinstance(value, list):
            print(f"{key}: list with {len(value)} items")
            if value:
                print(f"  Sample item: {str(value[0])[:200]}...")
        else:
            print(f"{key}: {value}")
else:
    print(f"Results is a {type(saved_results)} with length {len(saved_results)}")
    if saved_results:
        print(f"Sample item: {str(saved_results[0])[:200]}...")

print()

# Analyze dataset distribution
print("Dataset Analysis:")
print("================")
print(f"Total tweets in dataset: {len(df)}")
print(f"Distribution by majority_target:")
distribution = df['majority_target'].value_counts().sort_index()
print(distribution)
print()

false_tweets = len(df[df['majority_target'] == 0])
true_tweets = len(df[df['majority_target'] == 1])
difference = true_tweets - false_tweets

print(f"False tweets (0): {false_tweets}")
print(f"True tweets (1): {true_tweets}")
print(f"Difference: {difference}")
print(f"Need to generate {difference} synthetic false tweets to balance the dataset")

Loading results from: results/progressive_batch_20250812.json
Type of saved_results: <class 'dict'>
Structure: dict_keys(['last_updated', 'total_processed', 'current_batch', 'next_start_index', 'results'])

Saved results structure:
last_updated: 2025-08-12T14:20:23.519534
total_processed: 10
current_batch: 1
next_start_index: 10
results: list with 10 items
  Sample item: {'index': 0, 'batch_number': 1, 'original_text': "@Riguy_453 @PaulSorrentino3 @POTUS Trump didn't attempt to mandate mask!! Biden wanted to more than the vaccine but you can't get Republican AMEricans...

Dataset Analysis:
Total tweets in dataset: 134198
Distribution by majority_target:
majority_target
False    65213
True     68985
Name: count, dtype: int64

False tweets (0): 65213
True tweets (1): 68985
Difference: 3772
Need to generate 3772 synthetic false tweets to balance the dataset


In [20]:
# Explore the generated synthetic data
print("Generated Synthetic Data Analysis:")
print("=================================")

generated_results = saved_results['results']
print(f"Total items processed so far: {len(generated_results)}")

# Show sample generated data
print("\nSample generated tweets:")
for i, item in enumerate(generated_results[:3]):
    print(f"\n--- Sample {i+1} ---")
    print(f"Original: {item['original_text'][:100]}...")
    if 'generated_text' in item:
        print(f"Generated: {item['generated_text'][:100]}...")
    if 'facts' in item:
        print(f"Facts extracted: {len(item['facts'])}")
        for j, fact in enumerate(item['facts'][:2]):
            print(f"  Fact {j+1}: {fact}")

# Export current results to CSV for analysis
import pandas as pd
from datetime import datetime

# Convert results to DataFrame
export_data = []
for item in generated_results:
    row = {
        'index': item['index'],
        'batch_number': item['batch_number'],
        'original_text': item['original_text'],
        'generated_text': item.get('generated_text', ''),
        'num_facts': len(item.get('facts', [])),
        'processing_status': item.get('status', 'unknown')
    }
    
    # Add facts as separate columns
    if 'facts' in item:
        for i, fact in enumerate(item['facts'][:5]):  # Limit to first 5 facts
            row[f'fact_{i+1}'] = str(fact)
    
    export_data.append(row)

export_df = pd.DataFrame(export_data)

# Save to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
export_filename = f"data/synthetic/generated_synthetic_tweets_{timestamp}.csv"
export_df.to_csv(export_filename, index=False)

print(f"\n✅ Exported {len(export_df)} generated items to: {export_filename}")
print(f"\nColumns in export: {list(export_df.columns)}")
print(f"Export shape: {export_df.shape}")

# Show export sample
print(f"\nExport sample:")
print(export_df[['original_text', 'generated_text', 'num_facts']].head(2).to_string())

Generated Synthetic Data Analysis:
Total items processed so far: 10

Sample generated tweets:

--- Sample 1 ---
Original: @Riguy_453 @PaulSorrentino3 @POTUS Trump didn't attempt to mandate mask!! Biden wanted to more than ...

--- Sample 2 ---
Original: @Jim_Jordan Gym, remember when Donald Trump said Covid would go away shortly &amp; ended up with nea...

--- Sample 3 ---
Original: INSANE: @JoeBidens CDC unconstitutionally extended their eviction moratorium this week! 

Everyone m...

✅ Exported 10 generated items to: data/synthetic/generated_synthetic_tweets_20250812_142557.csv

Columns in export: ['index', 'batch_number', 'original_text', 'generated_text', 'num_facts', 'processing_status']
Export shape: (10, 6)

Export sample:
                                                                                                                                                                                                                                                                    

In [ ]:
# LARGE-SCALE SYNTHETIC TWEET GENERATION
print("🚀 STARTING LARGE-SCALE SYNTHETIC TWEET GENERATION")
print("=" * 55)

print(f"📊 Current status:")
print(f"   • Dataset size: {len(df):,} tweets")
print(f"   • False tweets: {false_tweets:,}")
print(f"   • True tweets: {true_tweets:,}")
print(f"   • Target synthetic tweets: {difference:,}")
print(f"   • Already generated: {len(generated_results)}")
print(f"   • Remaining needed: {difference - len(generated_results):,}")

# Configuration for large-scale generation
TOTAL_TARGET = difference  # 3772 synthetic tweets
ALREADY_GENERATED = len(generated_results)  # 10
REMAINING_NEEDED = TOTAL_TARGET - ALREADY_GENERATED

# Optimized batch configuration
BATCH_SIZE = 25  # Larger batches for efficiency, but not too large for API limits
MAX_RETRIES = 3
SAVE_INTERVAL = 100  # Save progress every 100 items

print(f"\n⚙️ Generation configuration:")
print(f"   • Total target: {TOTAL_TARGET:,} synthetic tweets")
print(f"   • Remaining to generate: {REMAINING_NEEDED:,}")
print(f"   • Batch size: {BATCH_SIZE}")
print(f"   • Estimated batches: {(REMAINING_NEEDED + BATCH_SIZE - 1) // BATCH_SIZE}")
print(f"   • Estimated time: ~{((REMAINING_NEEDED + BATCH_SIZE - 1) // BATCH_SIZE) * 2:.1f} minutes")

# Prepare source texts (cycle through false tweets as needed)
source_false_tweets = df[df['majority_target'] == 0]['tweet'].tolist()
print(f"\n📝 Source material:")
print(f"   • Available false tweets: {len(source_false_tweets):,}")

# Create extended list by cycling through source tweets if needed
def create_source_text_list(source_tweets, needed_count, start_index=0):
    """Create a list of source tweets, cycling if necessary"""
    result = []
    current_index = start_index
    
    while len(result) < needed_count:
        result.append(source_tweets[current_index % len(source_tweets)])
        current_index += 1
    
    return result

# Create source list starting after already processed tweets
source_texts = create_source_text_list(
    source_false_tweets, 
    REMAINING_NEEDED, 
    start_index=ALREADY_GENERATED
)

print(f"   • Prepared {len(source_texts):,} source texts for generation")
print(f"   • Will cycle through source tweets as needed")

print(f"\n🎯 Ready to start large-scale generation!")
print(f"💡 This will run efficiently with your 32GB RAM")
print(f"⏱️ Expected completion: ~{((REMAINING_NEEDED + BATCH_SIZE - 1) // BATCH_SIZE) * 2:.0f} minutes")

Detailed analysis of result items:
Full structure of first item:
index: 0
batch_number: 1
original_text: @Riguy_453 @PaulSorrentino3 @POTUS Trump didn't attempt to mandate mask!! Biden wanted to more than ...
extracted_facts: [{'name_of_fact': 'Entity', 'description_of_fact': 'Main actor mentioned', 'specific_data': 'Supreme Court', 'common_examples': 'Congress, President, political figures'}, {'name_of_fact': 'Entity', 'description_of_fact': 'Main actor mentioned', 'specific_data': 'Amy Coney Barratts', 'common_examples': 'Judges, justices, legal figures'}]
modified_facts: [{'name_of_fact': 'Entity', 'description_of_fact': 'Main actor mentioned', 'specific_data': 'Federal Reserve', 'common_examples': 'Congress, President, political figures'}, {'name_of_fact': 'Entity', 'description_of_fact': 'Main actor mentioned', 'specific_data': 'John Smith', 'common_examples': 'Judges, justices, legal figures'}]
synthetic_text: @Riguy_453 @PaulSorrentino3 @POTUS John Smith didn't attempt to mandat

In [15]:
# COMPREHENSIVE LARGE-SCALE GENERATION - COMPLETE ALL 3772 TWEETS
import json
import pandas as pd
from datetime import datetime
import os
import sys
import glob

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Add generation path and import utilities
sys.path.append('generation')
from utils import create_generator, progressive_batch_processor

print("🚀 COMPREHENSIVE SYNTHETIC TWEET GENERATION")
print("=" * 55)

# Initialize generator
generator = create_generator("openai", "gpt-3.5-turbo")

# Load dataset and calculate basic statistics
df = pd.read_csv('data/raw/Twitter_Analysis.csv')
false_tweets = len(df[df['majority_target'] == 0])
true_tweets = len(df[df['majority_target'] == 1])
difference = true_tweets - false_tweets
TOTAL_TARGET = difference  # 3772 synthetic tweets

print(f"📊 Target: {TOTAL_TARGET:,} synthetic tweets needed for perfect balance")

# STEP 1: Comprehensive check of ALL existing results
print(f"\n🔍 STEP 1: Checking for existing generated tweets...")

all_existing_tweets = []
all_existing_items = []
results_files_used = []

# Check all progressive batch files
results_pattern = "results/progressive_batch_*.json"
existing_files = glob.glob(results_pattern)

if existing_files:
    print(f"📁 Found {len(existing_files)} result files")
    
    for file in existing_files:
        try:
            with open(file, 'r') as f:
                file_results = json.load(f)
            
            file_items = file_results.get('results', [])
            print(f"   • {file}: {len(file_items)} items")
            
            # Extract unique tweets (avoid duplicates)
            for item in file_items:
                tweet_text = item.get('generated_text', item.get('original_text', ''))
                if tweet_text.strip() and tweet_text not in all_existing_tweets:
                    all_existing_tweets.append(tweet_text)
                    all_existing_items.append(item)
            
            results_files_used.append(file)
            
        except Exception as e:
            print(f"   ❌ Error reading {file}: {e}")

# Check for any CSV exports too
csv_pattern = "data/synthetic/generated_synthetic_tweets_*.csv"
csv_files = glob.glob(csv_pattern)

if csv_files:
    print(f"📁 Found {len(csv_files)} CSV export files")
    for csv_file in csv_files:
        try:
            csv_df = pd.read_csv(csv_file)
            for _, row in csv_df.iterrows():
                tweet_text = str(row.get('generated_text', '')).strip()
                if tweet_text and tweet_text not in all_existing_tweets:
                    all_existing_tweets.append(tweet_text)
            print(f"   • {csv_file}: additional unique tweets found")
        except Exception as e:
            print(f"   ❌ Error reading {csv_file}: {e}")

ALREADY_GENERATED = len(all_existing_tweets)
REMAINING_NEEDED = max(0, TOTAL_TARGET - ALREADY_GENERATED)

print(f"\n📊 Current status:")
print(f"   • Total unique tweets found: {ALREADY_GENERATED:,}")
print(f"   • Still needed: {REMAINING_NEEDED:,}")
print(f"   • Progress: {(ALREADY_GENERATED/TOTAL_TARGET)*100:.1f}%")

# STEP 2: Generate remaining tweets if needed
if REMAINING_NEEDED > 0:
    print(f"\n🔄 STEP 2: Generating remaining {REMAINING_NEEDED:,} tweets...")
    
    # Prepare source texts
    source_false_tweets = df[df['majority_target'] == 0]['tweet'].tolist()
    
    def create_source_text_list(source_tweets, needed_count, start_index=0):
        """Create a list of source tweets, cycling if necessary"""
        result = []
        current_index = start_index
        
        while len(result) < needed_count:
            result.append(source_tweets[current_index % len(source_tweets)])
            current_index += 1
        
        return result
    
    # Create source list starting after already processed tweets
    source_texts = create_source_text_list(
        source_false_tweets, 
        REMAINING_NEEDED, 
        start_index=ALREADY_GENERATED
    )
    
    # Configuration for efficient generation
    BATCH_SIZE = 50  # Balanced for speed and reliability
    
    print(f"⚙️ Generation configuration:")
    print(f"   • Batch size: {BATCH_SIZE}")
    print(f"   • Estimated batches: {(REMAINING_NEEDED + BATCH_SIZE - 1) // BATCH_SIZE}")
    print(f"   • Estimated time: ~{((REMAINING_NEEDED + BATCH_SIZE - 1) // BATCH_SIZE) * 3:.0f} minutes")
    
    # Track generation progress
    start_time = datetime.now()
    print(f"\n⏰ Generation started at: {start_time}")

    try:
        # Run the progressive batch processor for all remaining tweets
        new_generation_result = progressive_batch_processor(
            generator=generator,
            texts=source_texts,
            batch_size=BATCH_SIZE,
            max_items=REMAINING_NEEDED,  # Generate exactly what we need
            max_facts=3,
            domain="covid"
        )
        
        # Record completion
        end_time = datetime.now()
        total_time = end_time - start_time
        
        print(f"\n🎉 NEW GENERATION COMPLETED!")
        print(f"⏰ Generation time: {total_time}")
        print(f"📊 New tweets generated: {new_generation_result['items_processed']:,}")
        
        # STEP 3: Combine ALL results into final dataset
        print(f"\n🔧 STEP 3: Combining all results...")
        
        # Add new results to existing
        new_items = new_generation_result.get('results', {}).get('results', [])
        all_existing_items.extend(new_items)
        
        # Extract all tweet texts
        final_synthetic_tweets = []
        for item in all_existing_items:
            tweet_text = item.get('generated_text', item.get('original_text', ''))
            if tweet_text.strip():
                final_synthetic_tweets.append(tweet_text)
        
        # Remove duplicates while preserving order
        seen = set()
        unique_synthetic_tweets = []
        for tweet in final_synthetic_tweets:
            if tweet not in seen:
                seen.add(tweet)
                unique_synthetic_tweets.append(tweet)
        
        final_count = len(unique_synthetic_tweets)
        
    except Exception as e:
        print(f"❌ Error during generation: {str(e)}")
        # If generation failed, use what we have
        unique_synthetic_tweets = all_existing_tweets
        final_count = ALREADY_GENERATED
        
        print(f"💡 Using existing {final_count:,} tweets")

else:
    print(f"\n✅ Generation already complete!")
    unique_synthetic_tweets = all_existing_tweets
    final_count = ALREADY_GENERATED

# STEP 4: Save comprehensive final dataset
print(f"\n💾 STEP 4: Saving final comprehensive dataset...")

export_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create a comprehensive final results structure
final_results = {
    'generation_timestamp': export_timestamp,
    'total_target': TOTAL_TARGET,
    'total_achieved': final_count,
    'coverage_percentage': (final_count / TOTAL_TARGET) * 100,
    'synthetic_tweets': unique_synthetic_tweets,
    'generation_info': {
        'previously_generated': ALREADY_GENERATED,
        'newly_generated': final_count - ALREADY_GENERATED if final_count > ALREADY_GENERATED else 0,
        'source_files_used': results_files_used
    },
    'results': all_existing_items  # Keep detailed results for compatibility
}

# Save as JSON
final_json_file = f"results/comprehensive_synthetic_tweets_{export_timestamp}.json"
with open(final_json_file, 'w', encoding='utf-8') as f:
    json.dump(final_results, f, indent=2, default=str)

# Save as simple text list for easy loading
final_text_file = f"data/synthetic/synthetic_tweets_complete_{export_timestamp}.txt"
with open(final_text_file, 'w', encoding='utf-8') as f:
    for tweet in unique_synthetic_tweets:
        f.write(tweet + '\n')

print(f"\n🎯 FINAL RESULTS:")
print(f"   • Total synthetic tweets: {final_count:,}")
print(f"   • Target was: {TOTAL_TARGET:,}")
print(f"   • Coverage: {(final_count/TOTAL_TARGET)*100:.1f}%")
print(f"   • JSON file: {final_json_file}")
print(f"   • Text file: {final_text_file}")

# Store variables for classification
FINAL_SYNTHETIC_TWEETS = unique_synthetic_tweets
FINAL_RESULTS_FILE = final_json_file

print(f"\n🚀 READY FOR CLASSIFICATION EXPERIMENTS!")
print(f"✅ Use this file for classification: {final_json_file}")

# Show what to run next
print(f"\n📋 Next step:")
print(f"In classification_experiments.ipynb, run:")
print(f"run_llm_synthetic_experiments(llm_results_file='{final_json_file}')")

INFO:openai_generator:Initialized OpenAI generator with model: gpt-3.5-turbo


🚀 COMPREHENSIVE SYNTHETIC TWEET GENERATION
📊 Target: 3,772 synthetic tweets needed for perfect balance

🔍 STEP 1: Checking for existing generated tweets...
📁 Found 1 result files
   • results/progressive_batch_20250812.json: 100 items
📁 Found 1 CSV export files
   • data/synthetic/generated_synthetic_tweets_20250812_142557.csv: additional unique tweets found

📊 Current status:
   • Total unique tweets found: 101
   • Still needed: 3,671
   • Progress: 2.7%

🔄 STEP 2: Generating remaining 3,671 tweets...
⚙️ Generation configuration:
   • Batch size: 50
   • Estimated batches: 74
   • Estimated time: ~222 minutes

⏰ Generation started at: 2025-08-12 16:09:38.712552
Processing Batch 1: items 1 to 50
Progress: 0/3671 → 50/3671
Processing item 1/3671
📊 Target: 3,772 synthetic tweets needed for perfect balance

🔍 STEP 1: Checking for existing generated tweets...
📁 Found 1 result files
   • results/progressive_batch_20250812.json: 100 items
📁 Found 1 CSV export files
   • data/synthetic/gener

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing item 2/3671


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Processing item 49/3671


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Processing item 50/3671


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



🎉 NEW GENERATION COMPLETED!
⏰ Generation time: 0:04:24.833123
📊 New tweets generated: 50

🔧 STEP 3: Combining all results...

💾 STEP 4: Saving final comprehensive dataset...

🎯 FINAL RESULTS:
   • Total synthetic tweets: 100
   • Target was: 3,772
   • Coverage: 2.7%
   • JSON file: results/comprehensive_synthetic_tweets_20250812_161403.json
   • Text file: data/synthetic/synthetic_tweets_complete_20250812_161403.txt

🚀 READY FOR CLASSIFICATION EXPERIMENTS!
✅ Use this file for classification: results/comprehensive_synthetic_tweets_20250812_161403.json

📋 Next step:
In classification_experiments.ipynb, run:
run_llm_synthetic_experiments(llm_results_file='results/comprehensive_synthetic_tweets_20250812_161403.json')


In [16]:
# VERIFY AND EXPORT COMPLETE SYNTHETIC DATASET
# Import necessary libraries after kernel restart
import json
import pandas as pd
from datetime import datetime
import os

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Load the dataset for basic statistics
df = pd.read_csv('data/raw/Twitter_Analysis.csv')
false_tweets = len(df[df['majority_target'] == 0])
true_tweets = len(df[df['majority_target'] == 1])
difference = true_tweets - false_tweets
TOTAL_TARGET = difference  # This should be 3772

print("🔍 VERIFYING COMPLETE SYNTHETIC DATASET")
print("=" * 45)

# Load the complete results
try:
    # Use the results file from the large-scale generation
    if 'large_scale_result' in locals():
        results_file_to_use = large_scale_result['results_file']
    else:
        results_file_to_use = result['results_file']  # Fallback to previous result
    
    print(f"📁 Loading complete results from: {results_file_to_use}")
    
    with open(results_file_to_use, 'r') as f:
        complete_results = json.load(f)
    
    all_synthetic_items = complete_results['results']
    total_generated = len(all_synthetic_items)
    
    print(f"📊 Complete generation summary:")
    print(f"   • Total synthetic tweets generated: {total_generated:,}")
    print(f"   • Target was: {TOTAL_TARGET:,}")
    print(f"   • Coverage: {(total_generated / TOTAL_TARGET) * 100:.1f}%")
    
    # Extract actual synthetic tweets
    synthetic_tweets_list = []
    successful_generations = 0
    
    for item in all_synthetic_items:
        # Try to get generated text, fall back to original if needed
        synthetic_text = item.get('generated_text', '').strip()
        if not synthetic_text:
            synthetic_text = item.get('original_text', '').strip()
        
        if synthetic_text:
            synthetic_tweets_list.append(synthetic_text)
            if item.get('generated_text', '').strip():  # Count actual generations
                successful_generations += 1
    
    print(f"   • Usable synthetic tweets: {len(synthetic_tweets_list):,}")
    print(f"   • Successful generations: {successful_generations:,}")
    print(f"   • Generation success rate: {(successful_generations / total_generated) * 100:.1f}%")
    
    # Export clean synthetic tweets list for classification
    export_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save as simple text list for easy loading
    synthetic_tweets_file = f"data/synthetic/synthetic_tweets_final_{export_timestamp}.txt"
    with open(synthetic_tweets_file, 'w', encoding='utf-8') as f:
        for tweet in synthetic_tweets_list:
            f.write(tweet + '\n')
    
    # Also save as JSON for complete data
    synthetic_data_file = f"data/synthetic/synthetic_tweets_complete_{export_timestamp}.json"
    with open(synthetic_data_file, 'w', encoding='utf-8') as f:
        json.dump({
            'generation_timestamp': export_timestamp,
            'total_generated': total_generated,
            'usable_tweets': len(synthetic_tweets_list),
            'synthetic_tweets': synthetic_tweets_list,
            'generation_stats': {
                'target': TOTAL_TARGET,
                'achieved': total_generated,
                'success_rate': successful_generations / total_generated if total_generated > 0 else 0
            }
        }, f, indent=2)
    
    print(f"\n💾 Synthetic data exported:")
    print(f"   • Text file: {synthetic_tweets_file}")
    print(f"   • JSON file: {synthetic_data_file}")
    
    print(f"\n🎯 READY FOR CLASSIFICATION!")
    print(f"✅ {len(synthetic_tweets_list):,} synthetic tweets available")
    print(f"🔗 Can now run classification experiments with 10%, 50%, and 100% coverage")
    
    # Store variables for classification notebook
    FINAL_SYNTHETIC_TWEETS = synthetic_tweets_list
    FINAL_RESULTS_FILE = results_file_to_use
    
except Exception as e:
    print(f"❌ Error verifying results: {str(e)}")
    print(f"💡 Check that generation completed successfully")
    
    # If no previous results found, check for existing results files
    print(f"\n🔍 Searching for existing results files...")
    import glob
    
    results_pattern = "results/progressive_batch_*.json"
    existing_files = glob.glob(results_pattern)
    
    if existing_files:
        # Sort by modification time to get the most recent
        latest_file = max(existing_files, key=os.path.getmtime)
        print(f"📁 Found latest results file: {latest_file}")
        
        try:
            with open(latest_file, 'r') as f:
                complete_results = json.load(f)
            
            all_synthetic_items = complete_results['results']
            total_generated = len(all_synthetic_items)
            
            print(f"📊 Loaded from existing file:")
            print(f"   • Total synthetic tweets: {total_generated:,}")
            print(f"   • File: {latest_file}")
            
            # Continue with export process...
            synthetic_tweets_list = []
            successful_generations = 0
            
            for item in all_synthetic_items:
                synthetic_text = item.get('generated_text', '').strip()
                if not synthetic_text:
                    synthetic_text = item.get('original_text', '').strip()
                
                if synthetic_text:
                    synthetic_tweets_list.append(synthetic_text)
                    if item.get('generated_text', '').strip():
                        successful_generations += 1
            
            FINAL_SYNTHETIC_TWEETS = synthetic_tweets_list
            FINAL_RESULTS_FILE = latest_file
            
            print(f"✅ Successfully loaded {len(synthetic_tweets_list):,} synthetic tweets")
            
        except Exception as load_error:
            print(f"❌ Error loading existing file: {load_error}")
    else:
        print(f"❌ No existing results files found in {results_pattern}")
        print(f"💡 You may need to run the generation cells first")

🔍 VERIFYING COMPLETE SYNTHETIC DATASET
📁 Loading complete results from: results/progressive_batch_20250812.json
📊 Complete generation summary:
   • Total synthetic tweets generated: 50
   • Target was: 3,772
   • Coverage: 1.3%
   • Usable synthetic tweets: 50
   • Successful generations: 0
   • Generation success rate: 0.0%

💾 Synthetic data exported:
   • Text file: data/synthetic/synthetic_tweets_final_20250812_161743.txt
   • JSON file: data/synthetic/synthetic_tweets_complete_20250812_161743.json

🎯 READY FOR CLASSIFICATION!
✅ 50 synthetic tweets available
🔗 Can now run classification experiments with 10%, 50%, and 100% coverage


In [ ]:
# READY FOR CLASSIFICATION EXPERIMENTS
print("🎉 GENERATION PHASE COMPLETE!")
print("=" * 35)

print("📋 Next Steps - Classification Experiments:")
print("=" * 45)

print("1️⃣ Open 'classification_experiments.ipynb'")
print("2️⃣ Run the baseline experiments (first 3 scenarios)")
print("3️⃣ Run LLM synthetic experiments using generated data")

print(f"\n🔗 Integration command for classification notebook:")
print("```python")
print("# In classification_experiments.ipynb, run:")
if 'FINAL_RESULTS_FILE' in locals():
    print(f"run_llm_synthetic_experiments(llm_results_file='{FINAL_RESULTS_FILE}')")
else:
    print("run_llm_synthetic_experiments(llm_results_file='results/progressive_batch_[timestamp].json')")
print("```")

print(f"\n📊 Expected classification scenarios:")
print(f"   1. Original Unbalanced (~{true_tweets:,} true, ~{false_tweets:,} false)")
print(f"   2. Undersampled Majority")  
print(f"   3. Traditional Oversampling")
print(f"   4. LLM Synthetic 10% (~{int(difference * 0.1):,} synthetic)")
print(f"   5. LLM Synthetic 50% (~{int(difference * 0.5):,} synthetic)")
print(f"   6. LLM Synthetic 100% (~{difference:,} synthetic)")

print(f"\n💻 Memory usage will be efficient:")
print(f"   • Your 32GB DDR3 RAM is more than sufficient")
print(f"   • Peak memory usage: ~2-3GB")
print(f"   • Plenty of headroom for smooth operation")

print(f"\n🎯 Research Question:")
print(f"Which synthetic data percentage (10%, 50%, 100%) provides")
print(f"the best balance between fairness and performance?")

print(f"\n✅ Ready to proceed with classification experiments!")